In [1]:
import sqlite3
conn = sqlite3.connect('nominations.db')
schema = conn.execute('PRAGMA table_info(nominations);')
first_ten = conn.execute('SELECT * FROM nominations LIMIT 10;')
for item in schema:
    print(item)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [2]:
for item in first_ten:
    print(item)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


In [4]:
conn.execute('CREATE TABLE ceremonies (id integer PRIMARY KEY, Year integer, Host text);')
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
conn.executemany('INSERT INTO ceremonies (Year, Host) VALUES (?, ?)', years_hosts)

OperationalError: table ceremonies already exists

In [6]:
conn.execute('SELECT * FROM ceremonies LIMIT 10;').fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [7]:
conn.execute('PRAGMA table_info(ceremonies);').fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'Year', 'integer', 0, None, 0),
 (2, 'Host', 'text', 0, None, 0)]

In [8]:
conn.execute('PRAGMA foregin_keys=ON;')

In [10]:
nom_two = '''CREATE TABLE nominations_two 
(id integer PRIMARY KEY, 
category text, 
nominee text, 
movie text, 
character text, 
won integer, 
ceremony_id integer,
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id));
'''
conn.execute(nom_two)

In [11]:
query = '''
SELECT ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
FROM nominations
INNER JOIN ceremonies 
ON nominations.year == ceremonies.year;
'''
joined_nominations = conn.execute(query).fetchall()

In [12]:
insert_nominations_two = '''
INSERT INTO nominations_two 
(ceremony_id, category, nominee, movie, character, won)
VALUES (?,?,?,?,?,?);
'''
conn.executemany(insert_nominations_two, joined_nominations)

In [13]:
conn.execute('SELECT * FROM nominations_two LIMIT 5;').fetchall()

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1),
 (2,
  'Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  0,
  1),
 (3,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network',
  'Mark Zuckerberg',
  0,
  1),
 (4,
  'Actor -- Leading Role',
  'Colin Firth',
  "The King's Speech",
  'King George VI',
  1,
  1),
 (5,
  'Actor -- Leading Role',
  'James Franco',
  '127 Hours',
  'Aron Ralston',
  0,
  1)]

In [14]:
conn.execute('DROP TABLE nominations;')

In [15]:
conn.execute('ALTER TABLE nominations_two RENAME TO nominations')

In [16]:
movies = '''
CREATE TABLE movies (
id INTEGER PRIMARY KEY,
movie TEXT
);
'''
conn.execute(movies)

In [17]:
actors = '''
CREATE TABLE actors (
id INTEGER PRIMARY KEY,
actor TEXT
);
'''
conn.execute(actors)

In [18]:
movies_actors = '''
CREATE TABLE movies_actors (
id INTEGER PRIMARY KEY,
movie_id INTEGER REFERENCES movies(id),
actor_id INTEGER REFERENCES actors(id)
);
'''
conn.execute(movies_actors)

In [20]:
movies_data = conn.execute('SELECT movie FROM nominations').fetchall()
movies_insert = '''
INSERT INTO movies (movie) VALUES (?);
'''
conn.executemany(movies_insert, movies_data)

In [21]:
conn.execute('SELECT * FROM movies LIMIT 5;').fetchall()

[(1, 'Biutiful'),
 (2, 'True Grit'),
 (3, 'The Social Network'),
 (4, "The King's Speech"),
 (5, '127 Hours')]

In [23]:
actors_data = conn.execute('SELECT nominee FROM nominations').fetchall()
actors_insert = '''
INSERT INTO actors (actor) VALUES (?);
'''
conn.executemany(actors_insert, actors_data)

In [24]:
conn.execute('SELECT * FROM actors LIMIT 5;').fetchall()

[(1, 'Javier Bardem'),
 (2, 'Jeff Bridges'),
 (3, 'Jesse Eisenberg'),
 (4, 'Colin Firth'),
 (5, 'James Franco')]

In [25]:
conn.execute('SELECT * FROM movies_actors LIMIT 5;').fetchall()

[]

In [26]:
conn.execute('DROP TABLE movies_actors')

In [27]:
movies_actors = '''
CREATE TABLE movies_actors (
id INTEGER PRIMARY KEY,
movie_id INTEGER REFERENCES movies(id),
actor_id INTEGER REFERENCES actors(id)
);
'''
conn.execute(movies_actors)

In [28]:
conn.execute('SELECT * FROM movies_actors LIMIT 5;').fetchall()

[]

In [29]:
pairs_query = "select movie,nominee from nominations;"
movie_actor_pairs = conn.execute(pairs_query).fetchall()

join_table_insert = "insert into movies_actors (movie_id, actor_id) values ((select id from movies where movie == ?),(select id from actors where actor == ?));"
conn.executemany(join_table_insert,movie_actor_pairs)

In [30]:
conn.execute('SELECT * FROM movies_actors LIMIT 5;').fetchall()

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5)]